In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 68.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=e15f9381ea04124abab3a613d96afb48c78b3190a4d1ca55009de60b750216f4
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [31]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [6]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [7]:
df = spark.read.csv('players_21.csv', inferSchema = True, header = True)
df.show(10)

+---------+--------------------+-----------------+--------------------+---+----------+---------+---------+-----------+-------------------+--------------------+-----------+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+-------------+--------------------+---------+------------------+--------------------+-------------+------------------+-----------+----------+--------------------+---------------+--------------------+----+--------+-------+---------+---------+------+---------+-----------+----------+-----------+--------+--------------+--------------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+---------

In [8]:
df.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: string (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- nationality: string (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_rank: integer (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- wage_eur: integer (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- international_reputation: integer (nullable = true)
 |-- weak_foot: integer (nullable = true)
 |-- skill_moves: integer (nullable = true)
 |-- work_rate: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- real_face: st

1. La tabla de salida debe contener las siguientes columnas: short_name, long_name, age, height_cm, weight_kg, nationality, club_name, overall, potential, team_position

In [9]:
columnas = ["short_name", "long_name", "age", "height_cm", "weight_kg", "nationality", "club_name", "overall", "potential", "team_position"]
df = df.select(columnas)
df.show(10)

+-----------------+--------------------+---+---------+---------+-----------+-------------------+-------+---------+-------------+
|       short_name|           long_name|age|height_cm|weight_kg|nationality|          club_name|overall|potential|team_position|
+-----------------+--------------------+---+---------+---------+-----------+-------------------+-------+---------+-------------+
|         L. Messi|Lionel Andrés Mes...| 33|      170|       72|  Argentina|       FC Barcelona|     93|       93|          CAM|
|Cristiano Ronaldo|Cristiano Ronaldo...| 35|      187|       83|   Portugal|           Juventus|     92|       92|           LS|
|         J. Oblak|           Jan Oblak| 27|      188|       87|   Slovenia|    Atlético Madrid|     91|       93|           GK|
|   R. Lewandowski|  Robert Lewandowski| 31|      184|       80|     Poland|  FC Bayern München|     91|       91|           ST|
|        Neymar Jr|Neymar da Silva S...| 28|      175|       68|     Brazil|Paris Saint-Germain| 

In [10]:
def selecciona_columnas(df1):

  columnas = ["short_name", "long_name", "age", "height_cm", "weight_kg", "nationality", "club_name", "overall", "potential", "team_position"]
  df1 = df1.select(columnas)

  return df1

df = selecciona_columnas(df)

2. Agregar una columna player_cat que responderá a la siguiente regla (rank over Window particionada por nationality y team_position y ordenada por overall):


In [11]:
w = Window.partitionBy("nationality","team_position")
w

In [12]:
df = df.withColumn('p', F.dense_rank().over(w.orderBy(F.col('overall'))))
df.show()

+---------------+--------------------+---+---------+---------+-----------+-------------------+-------+---------+-------------+---+
|     short_name|           long_name|age|height_cm|weight_kg|nationality|          club_name|overall|potential|team_position|  p|
+---------------+--------------------+---+---------+---------+-----------+-------------------+-------+---------+-------------+---+
|  Luiz Henrique|Luiz Henrique Bes...| 23|      171|       64|     Brazil|      Moreirense FC|     66|       72|           RW|  1|
|  Ricardo Lopes|Ricardo Lopes Per...| 29|      183|       74|     Brazil|   Shanghai SIPG FC|     71|       71|           RW|  2|
|Adrián Burnabão|Adrián Guto Negrã...| 24|      176|       73|     Brazil|           Botafogo|     71|       71|           RW|  2|
|       Lourency|Lourency do Nasci...| 24|      174|       74|     Brazil|     Gil Vicente FC|     72|       77|           RW|  3|
|      Renatinho|Renato Ribeiro Ca...| 31|      167|       68|     Brazil|   Guangz

In [13]:
df = df.withColumn('player_cat',F.when(F.col('p').isin('1'),'A')
                       .when(F.col('p').isin('2'),'B')
                       .when(F.col('p').isin('3'),'C').otherwise('D')
                      )
df.show()

+---------------+--------------------+---+---------+---------+-----------+-------------------+-------+---------+-------------+---+----------+
|     short_name|           long_name|age|height_cm|weight_kg|nationality|          club_name|overall|potential|team_position|  p|player_cat|
+---------------+--------------------+---+---------+---------+-----------+-------------------+-------+---------+-------------+---+----------+
|  Luiz Henrique|Luiz Henrique Bes...| 23|      171|       64|     Brazil|      Moreirense FC|     66|       72|           RW|  1|         A|
|  Ricardo Lopes|Ricardo Lopes Per...| 29|      183|       74|     Brazil|   Shanghai SIPG FC|     71|       71|           RW|  2|         B|
|Adrián Burnabão|Adrián Guto Negrã...| 24|      176|       73|     Brazil|           Botafogo|     71|       71|           RW|  2|         B|
|       Lourency|Lourency do Nasci...| 24|      174|       74|     Brazil|     Gil Vicente FC|     72|       77|           RW|  3|         C|
|     

In [14]:
df = df.drop('p')
df.show(10)

+---------------+--------------------+---+---------+---------+-----------+----------------+-------+---------+-------------+----------+
|     short_name|           long_name|age|height_cm|weight_kg|nationality|       club_name|overall|potential|team_position|player_cat|
+---------------+--------------------+---+---------+---------+-----------+----------------+-------+---------+-------------+----------+
|  Luiz Henrique|Luiz Henrique Bes...| 23|      171|       64|     Brazil|   Moreirense FC|     66|       72|           RW|         A|
|  Ricardo Lopes|Ricardo Lopes Per...| 29|      183|       74|     Brazil|Shanghai SIPG FC|     71|       71|           RW|         B|
|Adrián Burnabão|Adrián Guto Negrã...| 24|      176|       73|     Brazil|        Botafogo|     71|       71|           RW|         B|
|       Lourency|Lourency do Nasci...| 24|      174|       74|     Brazil|  Gil Vicente FC|     72|       77|           RW|         C|
|      Renatinho|Renato Ribeiro Ca...| 31|      167|   

In [32]:
def agrega_player_cat(df1, column):

  w = Window.partitionBy("nationality","team_position")
  df1 = df1.withColumn('p', F.dense_rank().over(w.orderBy(F.col('overall'))))
  columnas = ["short_name", "long_name", "age", "height_cm", "weight_kg", "nationality", "club_name", "overall", "potential", "team_position"]
  df1 = df1.select(columnas)
  df1 = df1.withColumn(column,F.when(F.col('p').isin('1'),'A')
                       .when(F.col('p').isin('2'),'B')
                       .when(F.col('p').isin('3'),'C').otherwise('D')
                      )
  df1 = df1.drop('p')

  return df1

3. Agregaremos una columna potential_vs_overall con la siguiente regla: 
* Columna potential dividida por la columna overall

In [33]:
df = df.withColumn('potential_vs_overall', df['potential']/df['overall'])
df.show(10)

+---------------+--------------------+---+---------+---------+-----------+----------------+-------+---------+-------------+----------+--------------------+
|     short_name|           long_name|age|height_cm|weight_kg|nationality|       club_name|overall|potential|team_position|player_cat|potential_vs_overall|
+---------------+--------------------+---+---------+---------+-----------+----------------+-------+---------+-------------+----------+--------------------+
|  Luiz Henrique|Luiz Henrique Bes...| 23|      171|       64|     Brazil|   Moreirense FC|     66|       72|           RW|         A|  1.0909090909090908|
|  Ricardo Lopes|Ricardo Lopes Per...| 29|      183|       74|     Brazil|Shanghai SIPG FC|     71|       71|           RW|         B|                 1.0|
|Adrián Burnabão|Adrián Guto Negrã...| 24|      176|       73|     Brazil|        Botafogo|     71|       71|           RW|         B|                 1.0|
|       Lourency|Lourency do Nasci...| 24|      174|       74|  

In [ ]:
def agrega_potential_vs_overall(df1):

  df1 = df1.withColumn('potential_vs_overall', df1['potential']/df1['overall'])

  return df1
df = agrega_potential_vs_overall(df)

4. Filtraremos de acuerdo a las columnas player_cat y potential_vs_overall con las siguientes condiciones: 
* Si player_cat esta en los siguientes valores: A, B
* Si player_cat es C y potential_vs_overall es superior a 1.15
* Si player_cat es D y potential_vs_overall es superior a 1.25

In [16]:
df.filter((F.col('player_cat').isin('A','B'))).show(10)

+---------------+--------------------+---+---------+---------+-----------+-----------------+-------+---------+-------------+----------+--------------------+
|     short_name|           long_name|age|height_cm|weight_kg|nationality|        club_name|overall|potential|team_position|player_cat|potential_vs_overall|
+---------------+--------------------+---+---------+---------+-----------+-----------------+-------+---------+-------------+----------+--------------------+
|  Luiz Henrique|Luiz Henrique Bes...| 23|      171|       64|     Brazil|    Moreirense FC|     66|       72|           RW|         A|  1.0909090909090908|
|  Ricardo Lopes|Ricardo Lopes Per...| 29|      183|       74|     Brazil| Shanghai SIPG FC|     71|       71|           RW|         B|                 1.0|
|Adrián Burnabão|Adrián Guto Negrã...| 24|      176|       73|     Brazil|         Botafogo|     71|       71|           RW|         B|                 1.0|
|    C. Eriksson|  Carljohan Eriksson| 25|      191|      

In [17]:
df.filter((F.col('player_cat').isin('C')) & 
          (F.col('potential_vs_overall')>1.15)
          ).show(10)

+-----------------+--------------------+---+---------+---------+-----------+--------------------+-------+---------+-------------+----------+--------------------+
|       short_name|           long_name|age|height_cm|weight_kg|nationality|           club_name|overall|potential|team_position|player_cat|potential_vs_overall|
+-----------------+--------------------+---+---------+---------+-----------+--------------------+-------+---------+-------------+----------+--------------------+
|       F. Hörberg|       Felix Hörberg| 21|      180|       74|     Sweden|       Östersunds FK|     64|       74|          RWB|         C|             1.15625|
|B. Van Den Eynden|  Bas Van Den Eynden| 18|      186|       83|    Belgium|         KV Mechelen|     55|       73|          RES|         C|  1.3272727272727274|
|    J. Schelfhout|    Jordy Schelfhout| 19|      184|       79|    Belgium|         KV Oostende|     55|       69|          RES|         C|  1.2545454545454546|
|       L. Ritière|        L

In [18]:
df.filter((F.col('player_cat').isin('D')) & 
          (F.col('potential_vs_overall')>1.25)
          ).show(10)

+-------------+--------------------+---+---------+---------+-----------+--------------------+-------+---------+-------------+----------+--------------------+
|   short_name|           long_name|age|height_cm|weight_kg|nationality|           club_name|overall|potential|team_position|player_cat|potential_vs_overall|
+-------------+--------------------+---+---------+---------+-----------+--------------------+-------+---------+-------------+----------+--------------------+
|  A. Descotte|    Anthony Descotte| 16|      179|       75|    Belgium|Sporting de Charl...|     56|       79|          RES|         D|  1.4107142857142858|
|M. Parmentier|   Mathéo Parmentier| 17|      186|       75|    Belgium|            KAA Gent|     56|       76|          RES|         D|  1.3571428571428572|
|    T. Leysen|         Tobe Leysen| 18|      179|       83|    Belgium|            KRC Genk|     56|       75|          RES|         D|  1.3392857142857142|
|S. Van Dessel|     Stan Van Dessel| 18|      169|  

In [ ]:
def filtrar_A_B(df1):
  return df1.filter((F.col('player_cat').isin('A','B'))).show()

def filtrar_C(df1):
  return df1.filter((F.col('player_cat').isin('C')) & 
                    (F.col('potential_vs_overall')>1.15)
                    ).show()

def filtrar_D(df1):
  return df1.filter((F.col('player_cat').isin('D')) & 
                    (F.col('potential_vs_overall')>1.25)
                    ).show()

5. Agregar un parametro al archivo params que de ser 1 realice todos los pasos únicamente para los jugadores menores de 23 años y en caso de ser 0 que lo haga con todos los jugadores del dataset.

6. Por favor escriba la tabla resultante de los pasos anteriores particionada por la columna nationality, la salida debe estar escrita en formato parquet y debe usarse el método coalese(1) para obtener solo un archivo por partición.

In [30]:
df.coalesce(1).write.save('/content/drive/MyDrive/Parquet/one_file', format='parquet', mode='append', partitionBy='nationality')

In [ ]:
def escribe_tabla(df1):
  return df1.coalesce(1).write.save('/content/drive/MyDrive/Parquet/one_file', format='parquet', mode='append', partitionBy='nationality')

escribe_tabla(df)